In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns

# Inicialización

El primer paso que vamos a realizar es cargar las dos tablas en dos data frames distintos. 

In [ ]:
df_volvera = pd.read_csv("tp-2020-2c-train-cols1.csv")
df_volvera

In [ ]:
df_usuario = pd.read_csv("tp-2020-2c-train-cols2.csv")
df_usuario

En un primer análisis, podemos ver de ambas tablas que poseen un total de 801 filas. 

Consideramos hacer un join de las mismas, para operar y analizar la variable target(`volveria`) junto con los datos. 
Para ello utilizamos el campo `id_usuario` el cual es compartido.

Dejaremos en la variable `df_joined` las tablas unidas sin modificar, y usaremos `df` para ir realizando operaciones sobre la misma.

In [ ]:
df_joined = df_usuario.join(df_volvera.set_index('id_usuario'), on='id_usuario', how='inner').set_index('id_usuario')
df = df_joined.copy()
df

# Análisis de Datos

## Columnas con información duplicada

En este paso vamos a tratar de averiguar si tenemos información duplicada. El siguiente comando nos mostrara si existen filas exactamente iguales en la encuesta.

In [ ]:
df[df.duplicated(keep=False)]

Lo bueno de la verifcación anterior es que sabemos que no existio multiples respuestas exactamente iguales.

## Columnas con ausencia de información

El objetivo de este análisis es poder entender cuales son las columnas que poseen faltante de información.

In [ ]:
simbols = ["-","","_"]
for c in simbols:
    print("Column has simbol: '",c,"'")
    print(df.astype('str').eq(c).any())

Viendo el resultado anterior, sabemos que no existen caracteres especiales que puedan representar la ausencia de valor distinto a NaN.

A continuación, analizaremos las tablas con valores númericos. En la columna `edad` deberemos considerar como inválido cualquier valor menor o igual a cero. mientras que las columnas numéricas `amigos` `parientes` `precio_ticket` `volveria`, un valor menor a cero sería invalido.

In [ ]:
(df['edad'] <= 0).any()

In [ ]:
columnas_con_numeros = ['amigos', 'parientes', 'precio_ticket', 'volveria']
(df[columnas_con_numeros] < 0).any()

Por lo visto de analizar la información, no tenemos casos extraños de respuestas que requieran de un análisis particular. Sin embargo, nos queda una verificacición de reelevancia como lo es verificar valores en `null`.

In [ ]:
df.isnull().sum()

Como podemos ver de este último comando, la columna `fila` posee unos 624 nulls de los 801 valores. Adicionalmente, la segunda columna con poca información es la columna edad. A continuación calcularemos el porcentaje de que ausencia de valor de dichas columnas.

In [ ]:
(df.isnull().mean()*100).to_frame('porcentaje nulls')

Viendo estos valores, podemos observar que en la columna `fila` que representa la fila donde se sento el cliente al ver la pelicula tiene un 78% de ausencia de información. En cuanto a la columna `edad`, encontramos un 20% de faltante de información. 

Es importante mencionar tambien que la columna `nombre_sede` tiene algunos valores en null por lo que deberemos tomar una decisión de que hacer con dichas entradas. En este paso vamos consultar si tenemos alguna fila duplicada.

## Funciones para el Analisis de cada Columna

La función `porcentajeVolveria` nos servira para poder obtener los porcentajes que volverian basados en un agrupamiento.

In [ ]:
def porcentajeVolveria(group):
    return (group == 1).mean() * 100.0

## Análisis de la columna "nombre" y "numero_de_ticket"

Consideramos que el análisis de nombres y de los números de tickets son irrelevantes para determinar una relación que sirva a fiumark para determinar si la persona volveria ó no.

Analizar estas variables podría conducir a correlaciones espurias con la variable target.

## Analisis de la columna "Fila"

En este inciso analizaremos la columna `Fila`. Como vimos anteriormente, es una columna con muchos valores vacios. Es importante poder ver que hacer con dichos valores o si la columna tiene algo de información que nos pueda ser util. 

In [ ]:
volveria_por_fila = df.groupby(by='fila').agg({
        "volveria" : porcentajeVolveria,       
})
volveria_por_fila

In [ ]:
plt.figure(dpi=100)
sns.countplot(data=df, x='fila', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'},orient="vertical")
plt.ylabel("Cantidad")
plt.xlabel("Fila del Asiento donde vio Frozen 3")
plt.title("Volverian segun la fila")

Además del gran cantidad de ausencia de información acerca de dicha variable, lo que vemos es que la gran mayoría que contesto dijo que lo hizo en filas de adelante, lo que en principio nos dificulta poder hacer una diferencia entre los dos valores que tenemos que nos pueda determinar concretamente una relación entre el lugar donde se sento y la preferencia para volver.

Teniendo esto en cuenta, y que no podemos inferir un valor para reemplazar en las ausencias, vamos a proceder a descartar dicha columna de nuestros analisis.

In [ ]:
df.drop(columns=['fila'],inplace=True)
df

## Analisis de la columna "Edad"

Reutilizaremos la función definida anteriomente para analizar la columna edad.

In [ ]:
volveria_por_edad = df.groupby(by='edad').agg({
        "volveria" : porcentajeVolveria,       
})
volveria_por_edad

En un primer analisis sobre las edades, podemos observar valores decimales lo cual genera demasiados valores posibles para dicha variable. Para poder optimizar los calculos, vamos a generar una categoría agrupando de a 10 años tomando su digito mas significativo. Ejemplo, todos aquellos que respondieron que tienen entre 20 y 30 años, estaran categorizados como 2.

In [ ]:
def categorizarEdad(edad):
    if(np.isnan(edad)):
        return edad
    else:
        return int(round(edad/10,0))


df['edad_categorizada'] = df.edad.apply(categorizarEdad)
df

In [ ]:
plt.figure(dpi=120)
sns.countplot(data=df, x='edad_categorizada', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'},orient="vertical")
plt.ylabel("Cantidad")
plt.xticks(rotation='vertical')
plt.xlabel("Categoria de Edad")
plt.title("Volverian segun categoria de edad")

El grafico anterior nos muestra que para personas con edad menor a 20 años, existe ciertos valores que podrian indicar una preferencia por volver. Sin embargo, de 20 años en adelante, en cada categoría la mayoria dice que no volveria. Esto tambien lo podemos observar obteniendo los porcentajes que volverian segun cada categoría.

In [ ]:
df.groupby(by='edad_categorizada').agg({
        "volveria" : porcentajeVolveria
})

## Analisis de la columna "Genero"

Para el analisis por genero, lo primero que trataremos de observa es si existe algun tipo de preferencia entre volver o no dependiendo del mismo.

In [ ]:
plt.figure(dpi=100)
sns.countplot(data=df, x='genero', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'},orient="vertical")
plt.ylabel("Cantidad")
plt.xlabel("Genero")
plt.title("Volverian segun el Genero")

Como podemos observar, existe una relación muy fuerte entre el genero y las respuestas que dieron sobre si volverian o no, teniendo en los hombres una predeterminación a no volver y en las mujeres lo opuesto.

In [ ]:
df.groupby(by='genero').agg({
        "volveria" : porcentajeVolveria
})

Con estos ultimos datos, podemos ver que la tendencia de no regresar de los hombres es mas marcada que la de volver de las mujeres. Aún así, estos valores nos muestran una tendencia muy marcada para cada uno de los generos, lo que nos proporciona una buena variable para iniciar nuestra función de predicción.

## Anásisis de la columna "Amigos"

Aquí nos centraremos en ver si existe alguna influencia con la variable target: "volveria" respecto a la cantidad de amigos.

In [ ]:
plt.figure(dpi=120)
sns.countplot(data=df, x='amigos', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'})

plt.show()

Podemos observar que la cantidad de personas que **no volveria es importante para cuando la persona va sin amigos**, 
respecto a la cantidad de personas que volverían.

Mientras que para las personas que van con 1 amigo ó más, notamos que no hay diferencias marcadas como para tomar una determinación clara sobre si volveria al cine ó no.

A continuación los porcentajes en valor numérico para cada cantidad de amigos en el df:

In [ ]:
volveria_por_sede = df.groupby(by='amigos').agg({
        "volveria" : porcentajeVolveria,       
})
volveria_por_sede

## Análisis de la columna "Parientes"

En este caso haremos un análisis similar teniendo en cuenta también la cantidad de parientes respecto a la cantidad de personas que volverían(1) ó no(0)

In [ ]:
import seaborn as sns

plt.figure(dpi=130)
sns.countplot(data=df, x='parientes', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'})
plt.plot()
plt.show()

Podemos observar que la cantidad de personas que **no volveria es reelevante para cuando la persona va sin parientes**, 
respecto a la cantidad de personas que volverían para esa misma cantidad de parientes.

Mientras que para las personas que van con 1 pariente ó más, notamos que no hay diferencias marcadas como para tomar una determinación clara sobre si volveria al cine ó no.

A continuación una muestra de los porcentajes obtenidos para cada grupo de parientes:

In [ ]:
volveria_por_sede = df.groupby(by='parientes').agg({
        "volveria" : porcentajeVolveria,       
})
volveria_por_sede

### ¿Qué pasa cuando la persona concurre al cine acompañado ó no?

Luego de análizar las últimas 2 variables, vemos una tendencia a que la gente tiene una tendencia a no volver al cine en caso de ir sin amigos ó sin parientes, Entonces se nos ocurre la idea de analizar el dataframe construyendo una columna basándonos en si la persona fue acompañada al cine ó no, con el objetivo de ver si tienen la misma tendencia de resultados en el target: `volveria`, respecto a el análisis de amigos y parientes.

In [ ]:
def transformar_acompañado(amigos, parientes):
    return ((amigos > 0) | (parientes > 0))

df['acompañado'] = df.apply(lambda x: transformar_acompañado(x.amigos, x.parientes), axis=1)
plt.figure(dpi=120)
sns.countplot(data=df, x='acompañado', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'}, orient="vertical")
plt.show()

Esto nos da una noción de que el ir acompañado ayuda un poco pero no es un variable que nos sirva de mucho a la hora de decidir si volveria al cine.
Mientras que el hecho de ir sólo, hace que dificilmente vuelva y puede ser reelevante para futuros análisis y el armado del baseline.

Acontinuación se muestran los porcentajes obtenidos para esta nueva columna en el data frame:

In [ ]:
volveria_por_sede = df.groupby(by='acompañado').agg({
        "volveria" : porcentajeVolveria,       
})
volveria_por_sede

## Análisis de la columna "nombre_sede"

para este análisis usamos un gráfico de barras con el objetivo de cuantificar que cantidad de personas vuelve ó no al cine según la sede. para ello agrupo el data frame por sedes y luego ploteo con el target.

In [ ]:
plt.figure(dpi=120)
sns.countplot(data=df, x='nombre_sede', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'}, order=["fiumark_chacarita","fiumark_palermo","fiumark_quilmes"])
plt.title("Cantidad de personas que volvería/no volverían respecto a la sede")
plt.ylabel("Cantidad")
plt.xlabel("Sede")
plt.show()



Aquí podemos inferir que para la sede palermo la mayor parte de personas encuestadas no volvería, y en una gran cantidad respecto al total de usuarios encuestados(~800). Lo que nos da un pie a un posible futuro análisis junto a otras variables.

A continuación muestro los porcentajes de el target volveria en referencia a cada sede:

In [ ]:
volveria_por_sede = df.groupby(by='nombre_sede').agg({
        "volveria" : porcentajeVolveria,       
})
volveria_por_sede

## Análisis de la columna "tipo_de_sala"

Nuevamente analizamos la variable **tipo_de_sala**, discreta, de solo 3 tipos de sala: **normal, 3d y 4d**. ploteamos las cantidad y porcentajes en función de la variable target y obtuvimos los siguentes valores. 

In [ ]:
plt.figure(dpi=120)
sns.countplot(data=df, x='tipo_de_sala', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'} )
plt.title("Cantidad de personas que volverías/no volverían respecto al tipo de sala")
plt.ylabel("Cantidad")
plt.xlabel("Sala")
plt.show()

## Analisis de la columna "Precio Ticket"

Como previamente hicimos, analizaremos a grande razgos la variable en un principio para ver si existe algun categoría de precio que debamos analizar en particular.

In [ ]:
plt.figure(dpi=130)
sns.countplot(data=df, x='precio_ticket', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'},orient="vertical")
plt.ylabel("Cantidad")
plt.xlabel("Genero")
plt.title("Volverian segun el precio del ticket")

Como podemos observar, para los que su precio de ticket fue categorizado como `1`, las respuestas negativas son muy altas en comparación con las respuestas positivas. Esto nos inclina a analizar en parte para todos aquellos que tuvieron esa categoria, en comparación con los que tuvieron otra categoria de precio. Para esto, vamos a generar una nueva categoría que se llamara precio_1 y tendra un True o False si pertenece a dicha categoria.

In [ ]:
def categorizarPrecio(precio):
    return precio == 1


df['es_precio_ticket_1'] = df.precio_ticket.apply(categorizarPrecio)
df

In [ ]:
plt.figure(dpi=100)
sns.countplot(data=df, x='es_precio_ticket_1', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'},orient="vertical")
plt.ylabel("Cantidad")
plt.xlabel("¿Su ticket era de la categoria precio 1?")
plt.title("Volverian segun el precio del ticket")

Como ya vimos, para aquellos que fueron categorizados que abonaron el precio ticket `1` tienen una gran tendencia en la respuesta a no volver. Sin embargo, agrupando todas las otras categorias, vemos que es bastante similar los valores, con una leve inclinación a volver.

In [ ]:
df.groupby(by='es_precio_ticket_1').agg({"volveria" : porcentajeVolveria})

# Conclusión de los Análisis de las Variables Individuales

De las variables anteriores, hemos visto que la más marcada diferencia entre volveria o no y entre sus distintas sub-categorias fue la variable `genero`. Es por eso que vamos a tomar a esta primera variable para generar nuestra primera función de predicción.

# Función de Predicción (Accuracy)

La función `porcentualAcertado` nos permitira pasarle dos listados representando en el mismo orden los valores de si volvería o no retornando un valor porcentual de acierto.

In [ ]:
def porcentualAcertado(a,b):
    """Pre: recibe 2 listas de igual longitud siendo 
    a: la lista de las predicciones realizadas con la función de predicción 
    b: la lista de valores obtenidos de la variable target original.  
        Post: devolvió el porcentaje de aciertos de la función de predicción respecto de la original.
    """
    cantidad_acertada = 0
    total_usuarios = len(b)
    for i in range(total_usuarios):
        if(a[i] == b[i]):
            cantidad_acertada += 1
    return cantidad_acertada / total_usuarios

Luego definiremos la función `baseline` la cual tiene por objetivo recibir un data frame y así retornar el listado ordenado de si cumple o no para cada respuesta del formulario. Para esto utilizará la `funcionPrediccion` definida que determinara por fila si debe volver o no, retornando 0 para no volveria y 1 para volveria.

In [ ]:
def baseline(dataFrame):
    resultado = []
    for i, fila in dataFrame.iterrows():
        resultado.append(funcionPrediccion(fila))
    return resultado

Y por ultimo, vamos a definir la función `accuracy`, que nos retornara el porcentaje de aciertos que posee una función determinada para un data frame.

In [ ]:
def accuracy(dataFrame):
    aciertos = porcentualAcertado(baseline(dataFrame.drop(columns=['volveria'],inplace=False)), dataFrame['volveria'].to_list())
    return round(aciertos * 100, 2)

# Generación de la función de predicción

Primeramente vamos a tratar de analizar una función muy basica asumiendo que todos volverian

In [ ]:
def funcionPrediccion(fila):
     return 1
accuracy(df)

Como podemos observar, una función asi solo nos da un 38,33% de acierto. Este numero tampoco es al azar, ya que coincide con el porcentaje de volverian que habia en nuestro data frame. Para mejorar el valor predecido, podemos utilizar el negado de esta función asumiendo que todos no volverian, donde obtendremos el resultado complemento

In [ ]:
def funcionPrediccion(fila):
     return 0
accuracy(df)

De los analisis anteriores, esta vez vamos a generar una función de predicción utilizando la variable con mayor polarizacion que observamos que fue la variable Genero. En este caso, si nos encontramos con una mujer asumiremos que volvera, y en cambio para los hombres, el resultado será el opuesto.

In [ ]:
def funcionPrediccion(fila):
     return 1 if (fila['genero'] == 'mujer') else 0
accuracy(df)

Como podemos observar, ya solo haber realizado un filtrado por dicha variable, generamos una función de predicción capaz de acertar en un 78,9% en relación con la información original que teniamos, el cual es un mejor valor que asumir que ninguno volvia.

Realizaremos otras funciones de predición en base a los analisis superiores de cada variable individual como para revisar si existe alguna otra variable que nos presente un mejor valor.

In [ ]:
def funcionPrediccion(fila):
     return 1 if (fila['es_precio_ticket_1'] == False) else 0
accuracy(df)

In [ ]:
def funcionPrediccion(fila):
     return 1 if (fila['tipo_de_sala'] == '4D') else 0
accuracy(df)

In [ ]:
def funcionPrediccion(fila):
     return 1 if (fila['acompañado']) else 0
accuracy(df)

In [ ]:
def funcionPrediccion(fila):
     return 1 if (fila['nombre_sede'] == 'fiumark_chacarita') else 0
accuracy(df)

In [ ]:
def funcionPrediccion(fila):
     return 1 if (fila['tipo_de_sala'] == 'normal') else 0
accuracy(df)

Como pudimos observar de estos analisis, el que nos generó un mayor porcentaje de aciertos fue cuando nuestra función de predicción se baso simplemente en la variable `genero`.

## Correlación entre variables

A continuación a partir de los datos recabados de algunas de las columnas anteriores se nos ocurre a analizar si existen algunas relaciones entre las distintas variables que nos permita mejorar nuestra función de predicción. Para eso, trataremos de encontrar algún comportamiento común entre variables que no se relacione con lo que ya fuimos observando para cada una de las mismas individualmente.

### Genero en relación con el Precio del Ticket

Anteriormente observamos que las mujeres en su gran mayoría respondían que volverían, mientras que los hombres lo opuesto. Por otro lado, pudimos observar que en cuanto al precio pagado por el ticket, aquellos que su ticket tuvo el valor categorizado como `1`, no solo son el grupo mayoritario, sino que además, la gran mayoría de estos respondió que no volvería.

En los análisis siguientes trataremos de encontrar si existe o no una relación entre las variables. Para ello, lo que haremos es separar por genero y según si su precio de ticket corresponde a la categoría 1 o no.

In [ ]:
plt.figure(dpi=100)
sns.countplot(data=df[df.es_precio_ticket_1], x='genero', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'},orient="vertical")
plt.ylabel("Cantidad")
plt.xlabel("Genero")
plt.title("¿Volverian por Genero que tuvieron el precio de ticket 1?")

In [ ]:
plt.figure(dpi=100)
sns.countplot(data=df[df.es_precio_ticket_1 == False], x='genero', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'},orient="vertical")
plt.ylabel("Cantidad")
plt.xlabel("Categoria de Edad")
plt.title("¿Volverian por Genero que tuvieron otro precio de ticket que no fue el 1?")

De estos graficos podemos observar que en todos los escenarios, las mujeres volverían, mientras que los hombres no, independientemente de la si fueron categorizados en el precio que pagaron con el valor `1`. Es importante igual destacar, que el valor en las mujeres que volverian pagando dicha tarifa es mayor pero muy similar a las que no volverian. Dado este escenario que no altera lo que esperabamos, dejaremos el analizar la posibilidad de la influencia en el tipo de precio para más adelante.

### Genero en relación con el tipo de sala

Esta vez vamos a concentrarnos en si para algún tipo de sala, uno de los generos se comporta distinto a lo esperado, es decir, que las mujeres vuelvan en mayoría y que los hombres no lo hagan.

In [ ]:
plt.figure(dpi=100)
sns.countplot(data=df[df['genero'] == 'hombre' ], x='tipo_de_sala', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'},orient="vertical")
plt.ylabel("Cantidad")
plt.xlabel("Tipo de Sala")
plt.title("¿Cuál fue la respuesta para los hombres según el tipo de sala?")

In [ ]:
plt.figure(dpi=100)
sns.countplot(data=df[df['genero'] == 'mujer'], x='tipo_de_sala', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'},orient="vertical")
plt.ylabel("Cantidad")
plt.xlabel("Tipo de Sala")
plt.title("¿Cuál fue la respuesta para las mujeres según el tipo de sala?")

En el primer grafico no observamos ninguna diferencia con lo que veniamos viendo del análisis de cada variable, es decir, los hombres no volverían independientemente del tipo de sala. Sin embargo, en el segundo gráfico encontramos un escenario particular. Las mujeres para las salas 4D en su mayoría responden que no volverian. Esto nos da un indicio para mejorar nuestra función de predicción ya que deberemos considerar este escenario particular para las mujeres.

In [ ]:
def funcionPrediccion(fila):
    if (fila['genero'] == 'mujer') & (fila['tipo_de_sala'] != '4d'):
        return 1
    return 0
accuracy(df)

Esto nos permitio aumentar de 78,9% a un 79,53% de aciertos.

### Genero en relación con la sede

En este caso, vamos a evaluar si existe una relacion entre dichas variables.

In [ ]:
plt.figure(dpi=100)
sns.countplot(data=df[df['genero'] == 'hombre' ], x='nombre_sede', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'},orient="vertical")
plt.ylabel("Cantidad")
plt.xlabel("Sede")
plt.title("¿Cuál fue la respuesta para los hombres según la sede?")

In [ ]:
plt.figure(dpi=100)
sns.countplot(data=df[df['genero'] == 'mujer' ], x='nombre_sede', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'},orient="vertical")
plt.ylabel("Cantidad")
plt.xlabel("Sede")
plt.title("¿Cuál fue la respuesta para las mujeres según la sede?")

Para estos graficos, no encontramos nada distinto a lo que veníamos viendo. En todas las sedes las mujeres prefieren volver y los hombres no.

### Genero en relación con la edad

Para este escenario vamos a ver como se relaciona el genero con la edad utilizando inicialmente nuestra variable de categoría de edad.

In [ ]:
plt.figure(dpi=120)
data=df[df['genero'] == 'hombre' ]
sns.countplot(data=data, x='edad_categorizada', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'},orient="vertical")
plt.ylabel("Cantidad")
plt.xticks(rotation='vertical')
plt.xlabel("Categoria de Edad")
plt.title("¿Volverian los hombres dependiendo de la edad?")

In [ ]:
plt.figure(dpi=120)
data=df[df['genero'] == 'mujer' ]
sns.countplot(data=data, x='edad_categorizada', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'},orient="vertical")
plt.ylabel("Cantidad")
plt.xticks(rotation='vertical')
plt.xlabel("Categoria de Edad")
plt.title("¿Volverian las mujeres dependiendo de la edad?")

Como podemos observar encontramos un escenario particular que para la categoría de 0 para los hombres, es decir, hombres entre 0 y 10 años, su tendencia es la de volver. Esto es contrario a lo que esperabamos y deberíamos analizar mas dicha información. En este caso, lo que debemos hacer es encontrar el límite, para los cuales los hombres prefieren volver.

In [ ]:
plt.figure(dpi=120)
data=df[(df['genero'] == 'hombre') & (df['edad'] <= 20.0 ) ]
sns.countplot(data=data, x='edad', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'},orient="vertical")
plt.ylabel("Cantidad")
plt.xticks(rotation='vertical')
plt.xlabel("Categoría de Edad")
plt.title("¿Cómo se deglosa las respuestas de los hombres menores a 20 años?")

A continuación, nos interesa encontrar el punto de corte, para el cual existen mayores respuestas positivas que negativas.

In [ ]:
values = {}
data=df[df['genero'] == 'hombre' ]
for x in range(20,4,-1):
    data=data[data['edad'] <= x ]
    data_volverian = data[data['volveria'] == 1 ]
    y = data_volverian.value_counts().sum() / data.value_counts().sum() * 100
    values[x] = y
values

In [ ]:
lists = sorted(values.items())
x, y = zip(*lists)

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=[6.4 * 2 , 4.8], dpi=100)
axes.plot(x, y)
axes.set_title('Porcentaje de volvería vs Edad agrupada')
axes.set_xlabel('Edad [Años]')
axes.set_ylabel('Porcentaje de volvería[%]')

delimitador_bool = plt.axhline(y=50, color='r', linestyle='-', label='corte de acumulación')
plt.hlines(xmin=5, xmax=20, y=[35, 40, 45, 50, 55, 60, 65, 70], color='gray', linestyle='dotted')
plt.vlines(x=[6, 8, 10, 12, 14, 16, 18, 20], ymin=30, ymax=80, color='gray', linestyle='dotted')
plt.legend(handles=[delimitador_bool])


plt.show()

Como se puede observar, el punto de corte de acumulación es de 18 años. Esto quiere decir, que para todo hombre de que tenga menos de 18 años, tendra una tendencia positiva para volver

In [ ]:
def funcionPrediccion(fila):
    if ((fila['genero'] == 'mujer') & (fila['tipo_de_sala'] == '4d')):
        return 0
    if(fila['genero'] == 'mujer'):
        return 1
    if ((fila['genero'] == 'hombre') & (fila['edad'] < 18) ):
        return 1
    return 0
accuracy(df)

En este caso, mejoramos muy poco nuestra predicción pero pudimos incoporar una mejora determinando que los hombres menores a 18 prefieren volver.

### Genero en relación con si vino acompañado

En este caso, vamos a evaluar si existe una relación entre el `genero` y si vino `acompañado` a la hora de responder sobre si volverian.

In [ ]:
plt.figure(dpi=100)
sns.countplot(data=df[df['genero'] == 'hombre' ], x='acompañado', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'},orient="vertical")
plt.ylabel("Cantidad")
plt.xlabel("Sede")
plt.title("¿Cual fue la respuesta para los hombres que vinieron acompñados?")
plt.show()

In [ ]:
plt.figure(dpi=100)
sns.countplot(data=df[df['genero'] == 'mujer' ], x='acompañado', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'},orient="vertical")
plt.ylabel("Cantidad")
plt.xlabel("Sede")
plt.title("¿Cual fue la respuesta para las mujeres que vinieron acompñadas?")
plt.show()

Nuevamente, en este análisis no encontramos ningun resultado inesperado. Las mujeres prefieren volver y los hombres no.

## Correlación entre tres variables

En esta zona vamos a tratar de encontrar relaciones entre 3 variables. Dado que de la parte anterior aprendimos que las mujeres vuelven salvo para las salas 4D y los hombres no vuelven salvo aquellos que tienen menos de 18 años, estas dos variables las usaremos de base y buscaremos si una tercer variable puede afectar dichos resultados y darnos un indicio de como mejorar aún más nuestra función de predicción.

### Relaciones con el precio del ticket

In [ ]:
plt.figure(dpi=100)
data = df[ (df['genero'] == 'mujer') & (df['tipo_de_sala'] == '4d')]
sns.countplot(data=data, x='es_precio_ticket_1', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'},orient="vertical")
plt.ylabel("Cantidad")
plt.xlabel("Tipo de Sala")
plt.title("¿Cual fue la respuesta para las mujeres que asistieron a sala 4D segun el precio pagado?")
plt.show()

De este último gráfico, vemos que para las mujeres si pago el tipo de ticket `1`, indistintamente de haber asistido a la sala 4D, si deciden volver en su gran mayoría. Esto nos sirve para mejorar aún más nuestra función de predicción

In [ ]:
def funcionPrediccion(fila):
    if ((fila['genero'] == 'mujer') & (fila['tipo_de_sala'] == '4d') & (fila['es_precio_ticket_1'])):
        return 1
    if ((fila['genero'] == 'mujer') & (fila['tipo_de_sala'] == '4d')):
        return 0
    if(fila['genero'] == 'mujer'):
        return 1
    if ((fila['genero'] == 'hombre') & (fila['edad'] < 18) ):
        return 1
    return 0
accuracy(df)

Esto ya nos logro elevar nuestro valor de acierto a un 80,65%, y es el valor mas alto que hemos conseguido hasta el momento. Ahora nos falta por analizar si para las mujeres que no fueron a la sala 4D si el precio de ticket influencio.

In [ ]:
plt.figure(dpi=100)
data = df[ (df['genero'] == 'mujer') & (df['tipo_de_sala'] != '4d')]
sns.countplot(data=data, x='es_precio_ticket_1', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'},orient="vertical")
plt.ylabel("Cantidad")
plt.xlabel("Tipo de Sala")
plt.title("¿Cuál fue la respuesta para las mujeres que no asistieron a sala 4D según el precio pagado?")
plt.show()

En este caso, no obtuvimos información nueva.

Para continuar con los analisis, ahora vamos a trabajar con los hombres, y esta vez separando para los menores y para los mayores de 18 años.

In [ ]:
plt.figure(dpi=100)
data = df[ (df['genero'] == 'hombre') & (df['edad'] < 18)]
sns.countplot(data=data, x='es_precio_ticket_1', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'},orient="vertical")
plt.ylabel("Cantidad")
plt.xlabel("Tipo de Sala")
plt.title("¿Cuál fue la respuesta para las hombres menores de 18 años según el precio pagado?")
plt.show()

In [ ]:
plt.figure(dpi=100)
data = df[ (df['genero'] == 'hombre') & (df['edad'] >= 18)]
sns.countplot(data=data, x='es_precio_ticket_1', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'},orient="vertical")
plt.ylabel("Cantidad")
plt.xlabel("Tipo de Sala")
plt.title("¿Cuál fue la respuesta para las hombres mayores de 18 años según el precio pagado?")
plt.show()

Para estos dos graficos, no obtuvimos nueva información que nos sirva para mejorar nuestra función de predicción. Si cabe destacar que el resultado de hombres menores de 18 años y que no pagaron la tarfia del ticket 1 dio un valor igualado, pero esto no nos genera una tendencia inversa a lo que habiamos visto.

### Relaciones con el tipo de sala

In [ ]:
plt.figure(dpi=100)
data = df[ (df['genero'] == 'hombre') & (df['edad'] < 18)]
sns.countplot(data=data, x='tipo_de_sala', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'},orient="vertical")
plt.ylabel("Cantidad")
plt.xlabel("Tipo de Sala")
plt.title("¿Cuál fue la respuesta para las hombres menores de 18 años según el tipo de sala?")
plt.show()

De este gráfico observamos que los hombres menores de 18 años para las salas 4d prefieren no volver mientras que para las otras si. Esto se comporta distinto a lo esperado y nos sirve para nuevamente mejorar la función de predicción.

In [ ]:
def funcionPrediccion(fila):
    if ((fila['genero'] == 'mujer') & (fila['tipo_de_sala'] == '4d') & (fila['es_precio_ticket_1'])):
        return 1
    if ((fila['genero'] == 'mujer') & (fila['tipo_de_sala'] == '4d')):
        return 0
    if(fila['genero'] == 'mujer'):
        return 1
    if ((fila['genero'] == 'hombre') & (fila['edad'] < 18) & (fila['tipo_de_sala'] == '4d')): 
        return 0
    if ((fila['genero'] == 'hombre') & (fila['edad'] < 18) ):
        return 1
    return 0
accuracy(df)

Con este ultimo cambio, hemos llegado nuevamente al valor mas alto de predicción hasta el momento logrando un 81,65% de certeza utilizando los mismos datos de entrada.

In [ ]:
plt.figure(dpi=100)
data = df[ (df['genero'] == 'hombre') & (df['edad'] >= 18)]
sns.countplot(data=data, x='tipo_de_sala', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'},orient="vertical")
plt.ylabel("Cantidad")
plt.xlabel("Tipo de Sala")
plt.title("¿Cuál fue la respuesta para las hombres mayores de 18 años según el tipo de sala?")
plt.show()

En este ultimo gráfico, no encontramos ninguna diferencia con lo esperado de que los hombres mayores a 18 años deciden no volver.

Como ya habiamos analizado y encontrado una relacion entre las mujeres y el tipo de sala que nos sirvio para mejorar nuestro algortimo, no debemos volver a realizar dicha comparación.

### Relaciones con la variable acompañado

En este escenario analizaremos las relaciones entre mujeres, si asistieron o no a una sala 4d y si asistieron acompañados, entre los hombres, su edad en relacion a los 18 y si asistieron acompañados.

In [ ]:
plt.figure(dpi=100)
data = df[ (df['genero'] == 'mujer') & (df['tipo_de_sala'] == '4d')]
sns.countplot(data=data, x='acompañado', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'},orient="vertical")
plt.ylabel("Cantidad")
plt.xlabel("Tipo de Sala")
plt.title("¿Cual fue la respuesta para las mujeres que asistieron a sala 4D segun si estaban acompañadas?")
plt.show()

Esta función nos muestra que las mujeres que no fueron acompañadas y asistieron a una sala 4D, tienen una preferencia por volver. Realizaremos el cálculo de esta nueva funcion de predicción y compararemos si es mejor analizar por el acompañamiento o por si abonaron el tipo de ticket 1.

In [ ]:
def funcionPrediccion(fila):
    if ((fila['genero'] == 'mujer') & (fila['tipo_de_sala'] == '4d') & (fila['acompañado'] == False)):
        return 1
    if ((fila['genero'] == 'mujer') & (fila['tipo_de_sala'] == '4d')):
        return 0
    if(fila['genero'] == 'mujer'):
        return 1
    if ((fila['genero'] == 'hombre') & (fila['edad'] < 18) & (fila['tipo_de_sala'] == '4d')): 
        return 0
    if ((fila['genero'] == 'hombre') & (fila['edad'] < 18) ):
        return 1
    return 0
accuracy(df)

Dado que este resultado es el mas alto obtenido hasta el momento, encontramos que es mejor realizar esta comparación como tercer variable correlacionada para las mujeres que asistieron a salas 4D. Esto se da debido al volumen de casos de diferencia que en esta combinación fue mas alto que para la el precio de ticket 1.

In [ ]:
plt.figure(dpi=100)
data = df[ (df['genero'] == 'mujer') & (df['tipo_de_sala'] != '4d')]
sns.countplot(data=data, x='acompañado', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'},orient="vertical")
plt.ylabel("Cantidad")
plt.xlabel("Tipo de Sala")
plt.title("¿Cual fue la respuesta para las mujeres que no asistieron a sala 4D segun si estaban acompañadas?")
plt.show()

Este ultimo gráfico no nos dio ninguna nueva información util.

In [ ]:
plt.figure(dpi=100)
data = df[ (df['genero'] == 'hombre') & (df['edad'] < 18)]
sns.countplot(data=data, x='acompañado', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'},orient="vertical")
plt.ylabel("Cantidad")
plt.xlabel("Tipo de Sala")
plt.title("¿Cual fue la respuesta para las hombres menores de 18 años segun si vinieron acompañados?")
plt.show()

Si bien este graáico nos muestra que si no vino acompañado hay una preferencia negativa, el volumen que vemos es muy chico. Revisaremos si mejora la funcion de predicción o si en realidad la empeora.

In [ ]:
def funcionPrediccion(fila):
    if ((fila['genero'] == 'mujer') & (fila['tipo_de_sala'] == '4d') & (fila['acompañado'] == False)):
        return 1
    if ((fila['genero'] == 'mujer') & (fila['tipo_de_sala'] == '4d')):
        return 0
    if(fila['genero'] == 'mujer'):
        return 1
    if ((fila['genero'] == 'hombre') & (fila['edad'] < 18) & (fila['acompañado'] == False)): 
        return 0
    if ((fila['genero'] == 'hombre') & (fila['edad'] < 18) ):
        return 1
    return 0
accuracy(df)

Como vemos, esto genero un peor resultado por lo que procedemos a descartar esta información.

In [ ]:
plt.figure(dpi=100)
data = df[ (df['genero'] == 'hombre') & (df['edad'] >= 18)]
sns.countplot(data=data, x='acompañado', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'},orient="vertical")
plt.ylabel("Cantidad")
plt.xlabel("Tipo de Sala")
plt.title("¿Cuál fue la respuesta para las hombres mayores de 18 años según si vinieron acompañados?")
plt.show()

En este escenario, nuevamente, no volvimos a encontrar información útil, ya que conocíamos que los hombres mayores de 18 tenían tendencia a no volver.

### Relaciones con la sede

En este escenario analizaremos las relaciones entre mujeres, si asistieron o no a una sala 4d y la sede a la cuál asistieron, y entre los hombres, su edad en relación a los 18 y la sede donde asistieron.

In [ ]:
plt.figure(dpi=100)
data = df[ (df['genero'] == 'mujer') & (df['tipo_de_sala'] == '4d')]
sns.countplot(data=data, x='nombre_sede', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'},orient="vertical")
plt.ylabel("Cantidad")
plt.xlabel("Tipo de Sala")
plt.title("¿Cuál fue la respuesta para las mujeres que asistieron a sala 4D según la sede?")
plt.show()

Este resultado es interesante por dos motivos. En primer lugar, en dos sedes las mujeres informan que volverian por fuera de haber asistido a una sala 4d. Adicionalmente, los volumenes de diferencia parecen ser grandes para los volumenes que venimos manejando entre 3 variables, lo que nos da un indicio de tal vez haber dado con una variable que nos permita mejorar aun mas nuestro modelo de predicción.

In [ ]:
def funcionPrediccion(fila):
    if ((fila['genero'] == 'mujer') & (fila['tipo_de_sala'] == '4d') & (fila['nombre_sede'] != 'fiumark_palermo')):
        return 1
    if ((fila['genero'] == 'mujer') & (fila['tipo_de_sala'] == '4d')):
        return 0
    if(fila['genero'] == 'mujer'):
        return 1
    if ((fila['genero'] == 'hombre') & (fila['edad'] < 18) & (fila['tipo_de_sala'] == '4d')): 
        return 0
    if ((fila['genero'] == 'hombre') & (fila['edad'] < 18) ):
        return 1
    return 0
accuracy(df)

Efectivamente, esto nos llevo al mejor resultado hasta el momento, obteniendo un 83,15% de certeza en nuestra predicción.

In [ ]:
plt.figure(dpi=100)
data = df[ (df['genero'] == 'mujer') & (df['tipo_de_sala'] != '4d')]
sns.countplot(data=data, x='nombre_sede', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'},orient="vertical")
plt.ylabel("Cantidad")
plt.xlabel("Tipo de Sala")
plt.title("¿Cuál fue la respuesta para las mujeres que no asistieron a sala 4D según la sede?")
plt.show()

En este caso, no encontramos nueva información. Pasamos a hacer el analisis para los hombres.

In [ ]:
plt.figure(dpi=100)
data = df[ (df['genero'] == 'hombre') & (df['edad'] < 18)]
sns.countplot(data=data, x='nombre_sede', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'},orient="vertical")
plt.ylabel("Cantidad")
plt.xlabel("Tipo de Sala")
plt.title("¿Cuál fue la respuesta para las hombres menores de 18 años según la sede a la que asistieron?")
plt.show()

In [ ]:
def funcionPrediccion(fila):
    if ((fila['genero'] == 'mujer') & (fila['tipo_de_sala'] == '4d') & (fila['nombre_sede'] != 'fiumark_palermo')):
        return 1
    if ((fila['genero'] == 'mujer') & (fila['tipo_de_sala'] == '4d')):
        return 0
    if(fila['genero'] == 'mujer'):
        return 1
    if ((fila['genero'] == 'hombre') & (fila['edad'] < 18) & (fila['nombre_sede'] == 'fiumark_quilmes')): 
        return 0
    if ((fila['genero'] == 'hombre') & (fila['edad'] < 18) ):
        return 1
    return 0
accuracy(df)

Si bien encontramos nuevamente un valor distinto, el valor de acierto desendio de nuestro mejor resultado. Por lo que descartaremos esta información y mantendremos la anterior función de predicción.

In [ ]:
plt.figure(dpi=100)
data = df[ (df['genero'] == 'hombre') & (df['edad'] >= 18)]
sns.countplot(data=data, x='nombre_sede', hue='volveria', palette={0: '#FF5555', 1: '#44EE44'},orient="vertical")
plt.ylabel("Cantidad")
plt.xlabel("Tipo de Sala")
plt.title("¿Cuál fue la respuesta para las hombres mayores de 18 años según la sede a la que asistieron?")
plt.show()

Nuevamente, no encontramos información que nos mostrara un resultado imprevisto.

# Conclusión

Si bien el analisis puede continuar tratando de esta vez correlacionar 4 variables, decidimos no continuar debido a que superamos el valor solicitado por el cliente en cuanto al acierto minimo y debido a la complejidad extra de seguir correlacionado variables.

Cabe destacar que por fuera de los datos, la correlación que se dio para las mujeres, nos muestra un indicio de que la sala 4D de la sede de Fiumark Palermo podría no encontrarse en buenas condiciones lo que motiva a sus clientes a no querer volver a asistir. Esto también podríamos haberlo visto en los hombres ya que en su gran mayoría, los menores de 18 años si respondian que volverian, pero para los que asistieron a una sala 4D dicen que no. Esto nos abre las puertas para un proximo analisis donde quedaría observar si la sede palermo es la sede por la cual los menores de 18 años hombres decían que no volverían y que a su vez habían visto Frozen 3 en una sala 4D.

Por otro lado, como bien encontramos en nuestra variable principal, la temática de Frozen 3 nos da a entender que existe una fuerte tendencia por las mujeres a querer volver para ver Frozen 4, mientras que para los hombres no, y esto puede estar mas asociado con el género de la pelicula que con las instalaciones del lugar.

Como anteriormente mostramos, la función de predicción que mejor certeza nos genero es la siguiente:

In [ ]:
def funcionPrediccion(fila):
    if ((fila['genero'] == 'mujer') & (fila['tipo_de_sala'] == '4d') & (fila['nombre_sede'] != 'fiumark_palermo')):
        return 1
    if ((fila['genero'] == 'mujer') & (fila['tipo_de_sala'] == '4d')):
        return 0
    if(fila['genero'] == 'mujer'):
        return 1
    if ((fila['genero'] == 'hombre') & (fila['edad'] < 18) & (fila['tipo_de_sala'] == '4d')): 
        return 0
    if ((fila['genero'] == 'hombre') & (fila['edad'] < 18) ):
        return 1
    return 0
accuracy(df)